# SPeDE:

## Netwok analysis based on SPeDE USF matrix

### Script written by Charles Dumolin for LM-UGENT, 01-12-2020

In [1]:
# Import
import networkx as nx
import pandas as pd

from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, Legend, LegendItem
from bokeh.models.graphs import from_networkx

from bokeh.plotting import figure

In [2]:
## Read SPeDE USF-matrix file and metadatafile 

uni = pd.read_csv('SPeDE_validation_matrix_L_brevis.csv',sep=";",index_col=0)
data_raw = pd.read_csv('Metadata_L_brevis.csv',sep=";")
data = data_raw.to_dict('list')


# Transform the SPeDE USF-matrix to a link data frame (3 columns only):
links_uni = uni.stack().reset_index()
links_uni.columns = ['spectrumX', 'spectrumY','unique_features']
links_uni

# Keep only correlation where no unique feature could be observed
links_uni_filtered=links_uni.loc[(links_uni['unique_features'] == 0)]
links_uni_filtered


# Build the network and plot with Bokeh

G_uni=nx.from_pandas_edgelist(links_uni_filtered, 'spectrumX', 'spectrumY', 'unique_features')

nx.set_node_attributes(G_uni, pd.Series(data['isolate_number'],data['node']).to_dict(), 'isolate_number')
nx.set_node_attributes(G_uni, pd.Series(data['strain'],data['node']).to_dict(), 'strain')
nx.set_node_attributes(G_uni, pd.Series(data['reference'],data['node']).to_dict(), 'reference')
nx.set_node_attributes(G_uni, pd.Series(data['node_color'],data['node']).to_dict(), 'node_color')
nx.set_node_attributes(G_uni, pd.Series(data['OIU'],data['node']).to_dict(), 'OIU')
nx.set_node_attributes(G_uni, pd.Series(data['OIU_color'],data['node']).to_dict(), 'OIU_color')

nx.set_node_attributes(G_uni, pd.Series(data['node_line_color'],data['node']).to_dict(), 'node_line_color')
nx.set_node_attributes(G_uni, pd.Series(data['node_line_width'],data['node']).to_dict(), 'node_line_width')
nx.set_node_attributes(G_uni, pd.Series(data['node_size'],data['node']).to_dict(), 'node_size')
nx.set_node_attributes(G_uni, pd.Series(data['OIU_node_line_color'],data['node']).to_dict(), 'OIU_node_line_color')

G_uni.__dict__

# Show with Bokeh 
plot = figure(plot_width=1300, plot_height=850,title="", x_range=(-1.1,1.1), y_range=(-1.1,1.1),tools="box_zoom, reset,pan,undo,,redo,save")
#plot.title.text = "SPEDE_REF"

node_hover_tool = HoverTool(tooltips=[("isolate_number", "@isolate_number"),("R-number", "@strain"),("OIU", "@OIU"),("reference", "@reference"),("spectrum", "@index")])
plot.add_tools(node_hover_tool)

graph_renderer = from_networkx(G_uni, nx.spring_layout, iterations=1000, scale=0.5, center=(0,0))
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#797979", line_alpha=1, line_width=0.5, line_dash= "solid")


##STRAIN plot
#Change color of nodes
graph_renderer.node_renderer.glyph = Circle(size='node_size', fill_color='node_color', line_color='node_line_color', line_width='node_line_width')

plot.renderers.append(graph_renderer)
plot.axis.visible = False
plot.xgrid.visible = False
plot.ygrid.visible = False

#output_network
output_file("SPeDE_network_strain_colored.html")

show(plot)


## OIU plot
#Change color of nodes
graph_renderer.node_renderer.glyph = Circle(size='node_size', fill_color='OIU_color', line_color='OIU_node_line_color', line_width='node_line_width')

plot.renderers.append(graph_renderer)
plot.axis.visible = False
plot.xgrid.visible = False
plot.ygrid.visible = False

#output_network
output_file("SPeDE_network_OIU_colored.html")

show(plot)



